In [ ]:
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Conv2DTranspose, UpSampling2D
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
SHAPE = (300, 300)
SHAPE2 = (300, 300, 1)

In [ ]:
def add_noise(img):
    nf = 0.5
    norm = np.random.normal(loc=0.0, scale=1.0, size=img.shape)
    return img + nf + norm

In [ ]:
grounddata = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, dtype='float32')
xtrain = grounddata.flow_from_directory('data/train', target_size=SHAPE, batch_size=10, color_mode='grayscale', class_mode='categorical')
xtest = grounddata.flow_from_directory('data/test', target_size=SHAPE, batch_size=10, color_mode='grayscale', class_mode='categorical')

Found 5436 images belonging to 36 classes.
Found 5436 images belonging to 36 classes.


In [ ]:
noisydata = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, preprocessing_function=add_noise, dtype='float32')
xtrainnoisy = noisydata.flow_from_directory('data/train', target_size=(300, 300), batch_size=10, color_mode='grayscale', class_mode='categorical')
xtestnoisy = noisydata.flow_from_directory('data/test', target_size=(300, 300), batch_size=10, color_mode='grayscale', class_mode='categorical')

Found 5436 images belonging to 36 classes.
Found 5436 images belonging to 36 classes.


In [ ]:
classifier = Sequential()
classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=SHAPE2, padding='same'))
classifier.add(MaxPooling2D((2, 2), padding='same'))
classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
classifier.add(MaxPooling2D((2, 2), padding='same'))
classifier.add(Conv2D(16, kernel_size=(3, 3), activation='relu', strides=(2, 2), padding='same'))
classifier.add(MaxPooling2D((2, 2), padding='same'))
classifier.add(Conv2D(8, kernel_size=(3, 3), activation='relu', strides=(2, 2), padding='same'))
classifier.add(Conv2DTranspose(8, kernel_size=(3,3), activation='relu', strides=(2,2), padding='same'))
classifier.add(UpSampling2D((2, 2)))
classifier.add(Conv2DTranspose(16, kernel_size=(3,3), activation='relu', strides=(2,2), padding='same'))
classifier.add(UpSampling2D((2, 2)))
classifier.add(Conv2DTranspose(32, kernel_size=(3,3), activation='relu', strides=(2,2), padding='same'))
classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units=256, activation='relu'))
classifier.add(Dropout(0.50))
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.50))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=36, activation='softmax'))

In [ ]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 300, 300, 32)      320       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 150, 150, 32)      9248      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 38, 38, 16)        4624      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 19, 19, 16)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 10, 10, 8)        

In [34]:
classifier.fit(xtrainnoisy,
               epochs=100,
               batch_size=128,
               validation_data=xtestnoisy,
               shuffle=True)

Epoch 1/100
544/544 [==============================] - 1051s 2s/step - loss: 3.5848 - accuracy: 0.0239 - val_loss: 3.5836 - val_accuracy: 0.0278
Epoch 2/100
544/544 [==============================] - 984s 2s/step - loss: 3.5849 - accuracy: 0.0243 - val_loss: 3.5836 - val_accuracy: 0.0278
Epoch 3/100
544/544 [==============================] - 1009s 2s/step - loss: 3.5846 - accuracy: 0.0221 - val_loss: 3.5836 - val_accuracy: 0.0278
Epoch 4/100
544/544 [==============================] - 1121s 2s/step - loss: 3.5846 - accuracy: 0.0223 - val_loss: 3.5835 - val_accuracy: 0.0278
Epoch 5/100
544/544 [==============================] - 1344s 2s/step - loss: 3.5844 - accuracy: 0.0250 - val_loss: 3.5836 - val_accuracy: 0.0278
Epoch 6/100
544/544 [==============================] - 1764s 3s/step - loss: 3.5846 - accuracy: 0.0204 - val_loss: 3.5835 - val_accuracy: 0.0278
Epoch 7/100
544/544 [==============================] - 1694s 3s/step - loss: 3.5844 - accuracy: 0.0212 - val_loss: 3.5835 - val_acc

KeyboardInterrupt: 

In [35]:
'''Denoising and identification together not working
Might need to separate and try like before'''

'Denoising and identification together not working\nMight need to separate and try like before'